In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import plotly.graph_objects as go
from scipy.signal import find_peaks, savgol_filter, detrend
from scipy.stats import pearsonr

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score
from tools.analysis import *
from tools.outliers import MonteCarloOutliers
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import cross_val_predict, train_test_split
from sklearn.utils import shuffle

In [3]:
mpl.use('pgf')
mpl.rcParams.update({
    "pgf.texsystem": "pdflatex",  # Use pdflatex for rendering
    'font.family': 'serif',       # Use serif font family
    'text.usetex': True,          # Use LaTeX for text rendering
    'pgf.rcfonts': False,         # Don't setup fonts from rc params
    'pgf.preamble': r'\usepackage{amsmath,amsfonts,amssymb}\usepackage[T2A,T1]{fontenc}\usepackage[utf8]{inputenc}\usepackage[ukrainian]{babel}',  # Use Ukrainian Cyrillic font
    'font.size': 10               # Set font size
})

In [4]:
from boxsers.preprocessing import rubberband_baseline_cor

In [5]:
def corr_plot(X, y):
    r, p_val = pearsonr(X, y)
    
    model = LinearRegression()
    model.fit(X.reshape(-1, 1), y)
    
    x_range = np.linspace(min(X), max(X))
    predicted = model.predict(x_range.reshape(-1, 1))
    
    if p_val < 0.00001:
        text = f"$R = {r:.3f}$\n$p < 0.00001$\n$y = {model.coef_[0]:.5f}x + {model.intercept_:.5f}$"
    else:
        text = f"$R = {r:.3f}$\n$p = {p_val:.5f}$\n$y = {model.coef_[0]:.5f}x + {model.intercept_:.5f}$"
        
    plt.scatter(X, y, facecolor='w', edgecolor='k')
    plt.plot(x_range, predicted, color='r', linewidth=1.5)
    plt.xlabel("$y_1$")
    plt.ylabel("$y_2$")

    x_offset = (max(X) - min(X)) * 0.02
    y_offset = (max(y) - min(y)) * 0.02
    text_x = min(X)
    text_y = max(y)

    print(text)
    
    plt.text(text_x, text_y, text, verticalalignment='top')

In [6]:
def detrend_spectrum(spectrum, order):
    x = np.arange(len(spectrum))
    p = np.polyfit(x, spectrum, order)
    baseline = np.polyval(p, x)
    return spectrum - baseline

In [7]:
df = pd.read_csv('data/dataset.csv')
domain = pd.read_csv('data/domain.csv')
domain = np.ravel(domain)

#df = df[df['HbA1c'] < 10]
max_abs = df.drop(columns=['HbA1c', 'years']).apply(lambda row: True if row.max() > 0.3 else False, axis=1)
df = df[max_abs]
#df = df[df['HbA1c'] < 12]
print(len(df))

X = df.drop(columns=['HbA1c', 'years'])
y = df['HbA1c']
years = df['years']
X.columns = np.ravel(domain)

#X = X.T.apply(lambda col: np.ravel(rubberband_baseline_cor(col))).T
#X = X.apply(lambda row: row / np.linalg.norm(row), axis=1)
#X = X.apply(lambda row: (row - row.mean()) / row.std(), axis=1)
X = X.T.apply(lambda col: savgol_filter(col, window_length=55, polyorder=2, deriv=1)).T
#X = X.apply(lambda row: (row - row.mean()) / row.std(), axis=1)
X = X.apply(lambda row: detrend_spectrum(row, 1), axis=1)
#X = X.apply(lambda row: row / np.linalg.norm(row), axis=1)
#X = X.apply(lambda row: row / max(row.iloc[1000:]), axis=1)
#X.T.plot(legend=False)
X.T.plot(legend=False)

294


<Axes: >

In [8]:
plt.figure(figsize=(6.5, 4))
plt.plot(X.mean(axis=0), color='k', linewidth=1.3)
plt.xlim([400, 4000])
plt.xlabel("Хвильове число")
plt.ylabel("Поглинання")
plt.savefig("figures/first_derivative_ftir.pgf")

In [9]:
discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform', subsample=None, random_state=24)
categories = discretizer.fit_transform(y.to_numpy().reshape(-1, 1))

X_train, X_test, y_train, y_test = train_test_split(X.iloc[:, 1250:2700], y, test_size=0.3, stratify=categories, random_state=44)

fig, axs = plot_number_components(X_train, y_train, cv=10)
fig.set_size_inches(6.5, 6.5)
axs[0, 0].set_xlabel("Кількість компонент")
axs[0, 1].set_xlabel("Кількість компонент")
axs[0, 1].set_ylabel("PRESS")
axs[1, 0].set_xlabel("HbA1c виміряне")
axs[1, 0].set_ylabel("HbA1c передбачене")
axs[1, 1].set_xlabel("Зразок")
axs[1, 1].set_ylabel("Залишок")
plt.subplots_adjust(bottom=0.12)
fig.savefig("figures/training.pgf")

┌───────────────────────────┐
│        FIT RESULTS        │
├───────────────────────────┤
│ RMSE = 1.0077             │
│ R2 = 0.7178               │
│ R = 0.8508                │
│ Number of components = 12 │
└───────────────────────────┘


In [10]:
pls = PLSRegression(n_components=5)
pls.fit(X_train, y_train)
y_pred = pls.predict(X_test)
fig, axs = plsr_r2_plot(y_test, y_pred)
fig.set_size_inches(4, 3)
axs.set_xlabel("HbA1c виміряне")
axs.set_ylabel("HbA1c передбачене")
plt.subplots_adjust(bottom=0.13)
fig.savefig("figures/validation.pgf")

In [11]:
plt.plot(np.ravel(domain)[1250:2700], pls.coef_[0], color='k', linewidth=1.5)
plt.axhline(y=0, color='k', linestyle='--')
plt.xlabel("wavenumber")
plt.ylabel("regression coefficient")

Text(20.833291666666668, 0.5, 'regression coefficient')

In [12]:
db = y_test.copy()
db[y_test >= 6.5] = 1
db[y_test < 6.5] = 0
db

242    0.0
138    0.0
384    1.0
53     0.0
27     1.0
      ... 
379    1.0
114    0.0
451    1.0
118    0.0
256    0.0
Name: HbA1c, Length: 89, dtype: float64

In [13]:
db_pred = y_pred.copy()
db_pred[y_pred >= 6.5] = 1
db_pred[y_pred < 6.5] = 0
db_pred

array([0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
       1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0.,
       1., 0., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
       1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 1., 0., 0.])

In [14]:
accuracy_score(db, db_pred)

0.9213483146067416